Логистическая регрессия — один из видов линейных классификаторов. Одной из ее особенностей является возможность оценивания вероятностей классов, тогда как большинство линейных классификаторов могут выдавать только номера классов. 

Логистическая регрессия использует достаточно сложный функционал качества, который не допускает записи решения в явном виде (в отличие от, например, линейной регрессии). Тем не менее, логистическую регрессию можно настраивать с помощью градиентного спуска.

Мы будем работать с выборкой, содержащей два признака. Будем считать, что ответы лежат в множестве {-1, 1}.

In [28]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from typing import Tuple
from sklearn.metrics import roc_auc_score

# 1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [29]:
data = pd.read_csv('data-logistic.csv', header=None)
y = data[0]
X = data.loc[:,1:]

# 2. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. 
Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0). Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [39]:
def w1_iter(X: pd.DataFrame, y: pd.Series, w1: float, w2: float, k: float, C: float) -> float:
    l = len(y)
    Sum = 0
    for i in range(0, l):
        Sum += y[i] * X[1][i] * (1 - 1 / (1 + np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))
    
    return w1 + k / l * Sum - k * C * w1


def w2_iter(X: pd.DataFrame, y: pd.Series, w1: float, w2: float, k: float, C: float) -> float:
    l = len(y)
    Sum = 0
    for i in range(0, l):
        Sum += y[i] * X[2][i] * (1 - 1 / (1 + np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))
    
    return w2 + k / l * Sum - k * C * w2

In [ ]:
def gradient_descent(X: pd.DataFrame, y: pd.Series, w1: float=0.0, w2: float=0.0,
         k: float=0.1, C: float=0.0, precision: float=1e-5, max_iter: int=10000) -> Tuple[float, float]:
    for i in range(max_iter):
        w1_prev, w2_prev = w1, w2
        w1, w2 = w1_iter(X, y, w1, w2, k, C), w2_iter(X, y, w1, w2, k, C)
        if np.sqrt((w1_prev - w1) ** 2 + (w2_prev - w2) ** 2) <= precision:
            break

    return w1, w2

In [41]:
w1, w2 = gradient_descent(X,y)
w1_L2,w2_L2 = gradient_descent(X, y, C = 10)

# 3. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? 
Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)). 


In [42]:
def a(X: pd.DataFrame, w1: float, w2: float) -> pd.Series:
    return 1 / (1 + np.exp(- w1 * X[1] - w2 * X[2]))

y_sigm = a(X, w1, w2)
y_sigm_L2 = a(X, w1_L2, w2_L2)


f = open('answer.txt', 'w')
f.write(str(np.round(roc_auc_score(y, y_sigm), 3)))
f.write(' ')
f.write(str(np.round(roc_auc_score(y, y_sigm_L2), 3)))
f.close()